In [1]:
#!pip install torch~=1.7.0 torchvision pytorch-lightning

In [2]:
%matplotlib inline

In [3]:
!pip install lightly

     |████████████████████████████████| 267 kB 2.1 MB/s 
     |████████████████████████████████| 145 kB 20.5 MB/s 
     |████████████████████████████████| 112 kB 19.7 MB/s 
     |████████████████████████████████| 74 kB 3.0 MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141230 sha256=2bed16813beabb105c211d89161db2407c3653b943a64281ca476137574716fc
  Stored in directory: /root/.cache/pip/wheels/ca/33/b7/336836125fc9bb4ceaa4376d8abca10ca8bc84ddc824baea6c
Successfully built antlr4-python3-runtime


In [4]:
import torch
#import torch.nn as nn
import torchvision
import pytorch_lightning as pl
import lightly
from torchmetrics import F1

In [5]:
!nvidia-smi

Tue Oct 26 12:00:11 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.119.04   Driver Version: 450.119.04   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
import torch
torch.__version__

'1.9.1'

In [7]:
from torchvision.models import resnet18
from torch import nn
from torch.utils.data import DataLoader

In [8]:
import torchvision
from torchvision import datasets 
from torchvision.transforms import ToTensor

In [9]:
path_to_train ='../input/dogsandcats/Final Data/Train'
path_to_test ='../input/dogsandcats/Final Data/Val'

In [10]:
train_classifier_transforms = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Resize((60, 60)),
    torchvision.transforms.Normalize(
           mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
])
test_transforms = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Resize((60, 60)),
    torchvision.transforms.Normalize(
           mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
])

In [11]:
train_set = datasets.ImageFolder(root=path_to_train, transform= train_classifier_transforms)
test_set = datasets.ImageFolder(root= path_to_test , transform = test_transforms)

In [12]:
train_dl = DataLoader(train_set, batch_size=512, shuffle=True)
test_dl = DataLoader(test_set, batch_size=512)

In [13]:
import pytorch_lightning as pl
from pytorch_lightning.core.decorators import auto_move_data

In [14]:
class ResNetMNIST(pl.LightningModule):
  def __init__(self):
    super().__init__()
    self.model = resnet18(num_classes=10)
    self.loss = nn.CrossEntropyLoss()

  @auto_move_data
  def forward(self, x):
    return self.model(x)
  
  def training_step(self, batch, batch_no):
    x, y = batch
    logits = self(x)
    loss = self.loss(logits, y)
    return loss
  
  def configure_optimizers(self):
    return torch.optim.RMSprop(self.parameters(), lr=0.005)

/opt/conda/lib/python3.7/site-packages/pytorch_lightning/core/decorators.py:66: LightningDeprecationWarning: The `@auto_move_data` decorator is deprecated in v1.3 and will be removed in v1.5. Please use `trainer.predict` instead for inference. The decorator was applied to `forward`
  "The `@auto_move_data` decorator is deprecated in v1.3 and will be removed in v1.5."


In [15]:
model = ResNetMNIST()

In [16]:
trainer = pl.Trainer(
    gpus=1,
    max_epochs=110,
    progress_bar_refresh_rate=20
)
trainer.fit(model, train_dl)

/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:323: UserWarning: The number of training samples (30) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  f"The number of training samples ({self.num_training_batches}) is smaller than the logging interval"


Training: -1it [00:00, ?it/s]

In [17]:
trainer.save_checkpoint("resnet18_mnist.pt")

In [18]:
def get_prediction(x, model: pl.LightningModule):
  model.freeze() # prepares model for predicting
  probabilities = torch.softmax(model(x), dim=1)
  predicted_class = torch.argmax(probabilities, dim=1)
  return predicted_class, probabilities

In [19]:
from tqdm.autonotebook import tqdm

In [20]:
inference_model = ResNetMNIST.load_from_checkpoint("./resnet18_mnist.pt")

In [21]:
true_y, pred_y = [], []
for batch in tqdm(iter(test_dl), total=len(test_dl)):
  x, y = batch
  true_y.extend(y)
  preds, probs = get_prediction(x, inference_model)
  pred_y.extend(preds.cpu())

  0%|          | 0/4 [00:00<?, ?it/s]

In [22]:
from sklearn.metrics import classification_report

In [23]:
print(classification_report(true_y, pred_y, digits=2))

              precision    recall  f1-score   support

           0       0.85      0.69      0.76       402
           1       0.74      0.78      0.76       403
           2       0.69      0.83      0.76       404
           3       0.77      0.73      0.75       403

    accuracy                           0.76      1612
   macro avg       0.76      0.76      0.76      1612
weighted avg       0.76      0.76      0.76      1612

